In [45]:
import pandas as pd
import re
import difflib
import os
from difflib import SequenceMatcher
import subprocess
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
import statistics as s
from sklearn.utils import shuffle
import nltk
from nltk.tokenize import word_tokenize
import pickle
import numpy as np

In [46]:
df = pd.read_csv('df_centuries.csv')
df['old index'] = df.index.values.tolist()

In [47]:
print(len(df))

38798


In [48]:
df = shuffle(df).reset_index(drop=True)

In [49]:
# to try a smaller tryout dataset
df_1600s = df.loc[df['century'] == '1600s']
df_1700s = df.loc[df['century'] == '1700s']
df_1800s = df.loc[df['century'] == '1800s']
df_1900s = df.loc[df['century'] == '1900s']
#df = pd.concat([df_1600s, df_1700s, df_1800s, df_1900s]) small set turned off (else: use .head()). Whole set used.

In [50]:
total_pages_gt = 0
total_pages_ocr = 0

print('Before removal:')
for century in ['1600s', '1700s', '1800s', '1900s']:
    pages_gt = round(len(''.join(list(df.loc[df['century'] == century]['gt text'])))/1361)
    pages_ocr = round(len(''.join(list(df.loc[df['century'] == century]['ocr text'])))/1361)
    total_pages_gt += pages_gt
    total_pages_ocr += pages_ocr
    print(f'Total amount of pages {century} gt:', pages_gt)
    print(f'Total amount of pages {century} ocr:', pages_ocr)

print('Total pages gt:', total_pages_gt)
print('Total pages ocr:', total_pages_ocr )

Before removal:
Total amount of pages 1600s gt: 75242
Total amount of pages 1600s ocr: 73322
Total amount of pages 1700s gt: 50695
Total amount of pages 1700s ocr: 52902
Total amount of pages 1800s gt: 226653
Total amount of pages 1800s ocr: 226296
Total amount of pages 1900s gt: 25957
Total amount of pages 1900s ocr: 25488
Total pages gt: 378547
Total pages ocr: 378008


In [51]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

print('documents whole dataset:', len(df))
print('total pages if on average 0.5 page per document:', len(df)/2)

pages_per_document_list = []

for century in [df_1600s, df_1700s, df_1800s, df_1900s]:
    print('dataset:', namestr(century, globals())[0])
    print('documents:', len(century))
    print('percentage of total dataset:', len(century) / len(df))
    #print('assigned total pages:', (len(df)/2) * (len(century) / len(df)))
    print('assigned total pages:', (len(df)/2) / 4)
    pages_per_document = ((len(df)/2) / 4) / len(century)
    pages_per_document_list.append(pages_per_document)
    print('pages per document:', pages_per_document)
    
print(pages_per_document_list)
    
    

documents whole dataset: 38798
total pages if on average 0.5 page per document: 19399.0
dataset: df_1600s
documents: 34810
percentage of total dataset: 0.8972111964534254
assigned total pages: 4849.75
pages per document: 0.13932059752944556
dataset: df_1700s
documents: 1646
percentage of total dataset: 0.04242486726119903
assigned total pages: 4849.75
pages per document: 2.94638517618469
dataset: df_1800s
documents: 631
percentage of total dataset: 0.016263724934274963
assigned total pages: 4849.75
pages per document: 7.68581616481775
dataset: df_1900s
documents: 1711
percentage of total dataset: 0.04410021135110057
assigned total pages: 4849.75
pages per document: 2.8344535359438923
[0.13932059752944556, 2.94638517618469, 7.68581616481775, 2.8344535359438923]


In [52]:
reduced = []
reduced_pages = []
total_pages = []

new_gt_texts = []
new_ocr_texts = []

# reduce size of books
def reduce_size(text, century, ocr_or_gt, pages_per_document_list):
    if century == '1600s':
        pages = pages_per_document_list[0]
    elif century == '1700s':
        pages = pages_per_document_list[1]
    elif century == '1800s':
        pages = pages_per_document_list[2]
    elif century == '1900s':
        pages = pages_per_document_list[3]
    pages = pages # pick amount of pages
    one_page = 1361 # amount of characters on page
    characters =  one_page * pages
    characters = round(characters)
    #print('test:', text[0:characters])
    if len(text) > characters:
        first_pages = text[0:characters].split(".")
        #print('startthing:', first_pages)
        first_pages = '.'.join(first_pages[:-1])
        #print('first pages reduced:', first_pages)
        #reduced = 1
        if ocr_or_gt == 'ocr':
            total_pages_ocr = len(text) / one_page
            total_pages.append(total_pages_ocr)
            reduced_pages_ocr = total_pages_ocr - pages
            reduced_pages.append(reduced_pages_ocr)
            reduced.append(1)
    else:
        
        first_pages = text
        #reduced = 0
        if ocr_or_gt == 'ocr':
            total_pages_ocr = len(text) / one_page
            total_pages.append(total_pages_ocr)
            reduced_pages.append(0)
            reduced.append(0)
        #print('first pages not reduced:', first_pages)
    #print(type(first_pages))
    #print('end:', first_pages, reduced)
    
    if ocr_or_gt == 'ocr':
        new_ocr_texts.append(first_pages)
    elif ocr_or_gt == 'gt':
        new_gt_texts.append(first_pages)

df['gt text org'] = df['gt text']
df['ocr text org'] = df['ocr text'] 

#df['gt text'], df['reduced'] = df['gt text'].apply(lambda x: reduce_size(x))
#df['ocr text short'], df['reduced'] = df['ocr text'].apply(lambda x: reduce_size(x))
#df['gt text org'] = df['gt text']
#df['ocr text org'] = df['ocr text']


for index, row in df.iterrows():
    reduce_size(row['gt text'], row['century'], 'gt', pages_per_document_list)
    reduce_size(row['ocr text'], row['century'], 'ocr', pages_per_document_list)

df['gt text'] = new_gt_texts
df['ocr text'] = new_ocr_texts
    
    
total_pages_gt = 0
total_pages_ocr = 0
    
for century in ['1600s', '1700s', '1800s', '1900s']:
    pages_gt = round(len(''.join(list(df.loc[df['century'] == century]['gt text'])))/1361)
    pages_ocr = round(len(''.join(list(df.loc[df['century'] == century]['ocr text'])))/1361)
    total_pages_gt += pages_gt
    total_pages_ocr += pages_ocr
    print(f'Total amount of pages {century} gt:', pages_gt)
    print(f'Total amount of pages {century} ocr:', pages_ocr)

print('Total pages gt:', total_pages_gt)
print('Total pages ocr:', total_pages_ocr )

Total amount of pages 1600s gt: 1757
Total amount of pages 1600s ocr: 2339
Total amount of pages 1700s gt: 2029
Total amount of pages 1700s ocr: 1864
Total amount of pages 1800s gt: 4683
Total amount of pages 1800s ocr: 4692
Total amount of pages 1900s gt: 4491
Total amount of pages 1900s ocr: 4495
Total pages gt: 12960
Total pages ocr: 13390


In [53]:
# check if ocr text and gt text never got mixed up
same_texts = 0
different_texts = 0
list_gt = list(df['gt text'])
list_ocr = list(df['ocr text'])
for i in range(len(list_gt)):
    if list_gt[i] == list_ocr[i]:
        same_texts += 1
    elif list_gt[i] != list_ocr[i]:
        different_texts +=1
# the texts should in the vast majority of cases be different
print('same texts:', same_texts)
print('different texts:', different_texts)

same texts: 4264
different texts: 34534


In [54]:
# percentage of documents reduced
print('Percentage of docs reduced:', reduced.count(1)/len(reduced))
# percentage of dataset removed
print('Percentage of dataset removed:', sum(reduced_pages)/sum(total_pages))


reduced_count_df = pd.DataFrame({'reduced': reduced, 'source': df['source'], 'reduced pages': reduced_pages, 'total pages': total_pages, 'docs': [1]*len(df)})
sums = reduced_count_df[['reduced', 'source', 'reduced pages', 'docs']].groupby('source').sum()
means = reduced_count_df[['total pages', 'source', 'reduced pages']].groupby('source').mean()
sums['percentage reduced'] = sums['reduced'] / sums['docs']
print('sum:')
print(sums)
print('mean:')
print(means)
without_books = reduced_count_df.loc[reduced_count_df['source'] != 'dbnl']
without_books_removed = sum(list(without_books['reduced']))/ sum(list(without_books['docs']))
without_books['percentage removed'] = without_books['reduced pages'] / without_books['total pages']
print('Percentage of dataset outside of books removed pages:', s.mean(list(without_books['percentage removed'])))
print('Percentage of dataset outside of books reduced documents:', without_books_removed)

Percentage of docs reduced: 0.9254085262126914
Percentage of dataset removed: 0.9577041808251007
sum:
            reduced  reduced pages   docs  percentage reduced
source                                                       
Meertens      33756   68509.747841  34808            0.969777
anp              25       8.238588    204            0.122549
books             5       8.256318   1567            0.003191
dbnl            219  269326.544521    219            1.000000
newspapers     1899   24167.392482   2000            0.949500
mean:
            total pages  reduced pages
source                                
Meertens       2.106047       1.968218
anp            1.746211       0.040385
books          1.051324       0.005269
dbnl        1236.968000    1229.801573
newspapers    15.900721      12.083696
Percentage of dataset outside of books removed pages: 0.7500430728965978
Percentage of dataset outside of books reduced documents: 0.9249850955182871


/home/nvanthof/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [55]:
df_17th = df[df['century'] == '1600s']
df_18th = df[df['century'] == '1700s']
df_19th = df[df['century'] == '1800s']
df_20th = df[df['century'] == '1900s']


In [56]:
#datasets = [df_17th, df_18th, df_19th, df_20th] #datasets[0] == df_17th etc.
#datasets = [df_17th.head(), df_18th.head(), df_19th.head(), df_20th.head()]
#print(len(df_17th), len(df_18th),len(df_19th),len(df_20th))

In [57]:
# column selection
print(df.columns)
#df = df[['identifier', 'gt text', 'ocr text', 'CER', 'WER', 'source', 'word count gt',
#       'word count ocr', 'century']]

Index(['Unnamed: 0', 'identifier', 'gt text', 'ocr text', 'CER', 'WER',
       'WER (order independent)', 'dictionary lookup gt',
       'dictionary lookup ocr', 'jaccard_coefficient',
       'levenshtein_dist_normalized', 'source', 'word count gt',
       'word count ocr', 'year', 'century', 'old index', 'gt text org',
       'ocr text org'],
      dtype='object')


In [58]:
def preprocessing(text):
    text = re.sub("[^a-zA-Z0-9-. ]+", '', text) # remove punctuation except for hyphens and dots
    text = text.lower() # lowercase
    return text

# preprocess identifier
df['identifier'] = df['identifier'].apply(lambda x: str(x))
df['identifier'] = df['identifier'].apply(lambda x: x.replace('_', '.'))
df['identifier'] = df['identifier'].apply(lambda x: x.replace(':', '.'))
#df['identifier'] = df['identifier'].apply(lambda x: x + '_None')
# preprocess OCR and GT
df['gt text'] = df['gt text'].apply(lambda x: preprocessing(x))
df['ocr text'] = df['ocr text'].apply(lambda x: preprocessing(x))



In [59]:
# create parts for cross-validation, train, validation oand test set
df_17th = df[df['century'] == '1600s']
df_18th = df[df['century'] == '1700s']
df_19th = df[df['century'] == '1800s']
df_20th = df[df['century'] == '1900s']

datasets = [df_17th, df_18th, df_19th, df_20th]
# create a list of train, validation and test set

train = pd.DataFrame()
val = pd.DataFrame()
test = pd.DataFrame()
for dataset in datasets:
    splits = np.array_split(dataset, 5)
    train_sub = splits[:3]
    train_sub = pd.concat(train_sub)
    val_sub = splits[3]
    test_sub = splits[4]
    train = pd.concat([train, train_sub])
    val = pd.concat([val, val_sub])
    test = pd.concat([test, test_sub])

train_indices = train.index.values.tolist() 
val_indices = val.index.values.tolist()  
test_indices = test.index.values.tolist()  

for index, row in df.iterrows():
    if index in train_indices:
        df.at[index, 'set'] = 'train'
    elif index in val_indices:
        df.at[index, 'set'] = 'val'
    elif index in test_indices:
        df.at[index, 'set'] = 'test'


In [60]:
def fuzzy_matching(row): # create files with matched sentences
    #save_path = r"C:\Users\Gebruiker\Desktop\Thesis\Documenten\Documenten"
    
    ID = row['identifier']
    page = 'None'
    
    # split GT and OCR-output of each row into sentences (based on full stops).
    gt_sentences = row['gt text'].split(".")
    ocr_sentences = row['ocr text'].split(".")

    match_list = []

    for sentence in gt_sentences:
        for match in ocr_sentences:
            if SequenceMatcher(None, sentence, match).ratio() > 0.75:
                match_list.extend([[sentence, match]])
                
    matched_gt_sentences = []
    matched_ocr_sentences = []

    for sentence in match_list:
        if len(sentence[0]) or len(sentence[1]) > 0:
            matched_gt_sentences.append(sentence[0])
            matched_ocr_sentences.append(sentence[1])
    
    # .join() with lists
    separator = '.'
    doc_GT = separator.join(matched_gt_sentences)
    doc_OCR = separator.join(matched_ocr_sentences)
    
    #return matched_gt_sentences, matched_ocr_sentences
    return doc_GT, doc_OCR

In [61]:
jar_file = "ocrevalUAtion-1.3.4-jar-with-dependencies.jar"

def evaluation(index, row, doc_GT, doc_OCR):
    ID = row['identifier']
    page = 'None'
    
    doc_OCR = doc_OCR.replace('.', '')
    doc_OCR = re.sub(' +', ' ', doc_OCR)
    filename_ocr = f"{ID}_{page}_OCR.txt"
    #file_ocr = open(os.path.join(save_path, filename),"w+", encoding="utf-8")
    file_ocr = open(filename_ocr,"w+", encoding="utf-8")
    file_ocr.write(doc_OCR)
    file_ocr.close()
    
    doc_GT = doc_GT.replace('.', '')
    doc_GT = re.sub(' +', ' ', doc_GT)
    filename_gt = f"{ID}_{page}_GT.txt"
    #file_gt = open(os.path.join(save_path, filename),"w+", encoding="utf-8")
    file_gt = open(filename_gt,"w+", encoding="utf-8")
    file_gt.write(doc_GT)
    file_gt.close()
    
    #output = ID + '_' + page + ".html"
    output = f"{ID}_{page}.html"
    
    #process = subprocess.call("/home/nvanthof/jdk-16.0.1/bin/java -cp " + jar_file  + " eu.digitisation.Main -gt " + filename_gt + " -ocr "+ filename_ocr +" -o " + output + "")
    #os.system("/home/nvanthof/jdk-16.0.1/bin/java -cp /home/nvanthof/ocrevalUAtion-1.3.4-jar-with-dependencies.jar eu.digitisation.Main -gt /home/nvanthof/ddd.010728187.mpeg21.a0005_None_GT.txt -ocr /home/nvanthof/ddd.010728187.mpeg21.a0005_None_OCR.txt  -o /home/nvanthof/OUTPUT2.html")
    command = f"/home/nvanthof/jdk-16.0.1/bin/java -cp /home/nvanthof/ocrevalUAtion-1.3.4-jar-with-dependencies.jar eu.digitisation.Main -gt /home/nvanthof/{filename_gt} -ocr /home/nvanthof/{filename_ocr}  -o /home/nvanthof/{output}"
    #command = f"/usr/bin/java -cp /home/nynkegpu/ocrevalUAtion-1.3.4-jar-with-dependencies.jar eu.digitisation.Main -gt /home/nynkegpu/{filename_gt} -ocr /home/nynkegpu/{filename_ocr}  -o /home/nynkegpu/{output}"
    os.system(command)
    sleep(5)
    
    soup = BeautifulSoup(open(output, encoding='utf-8'))
    table = soup.find("table", attrs={'border': '1'})
    # Split the filename, and extract the identifier and pagenr together as identifier 
    # Find the first table (this is the table in which the scores are stored)
    # Find the tags in which 'CER', 'WER', and 'WER (order independent)' are stored and take the next tag to get the score 
    cer = table.find('td', text='CER')
    cerScore = cer.findNext('td')
    wer = table.find('td', text='WER')
    werScore = wer.findNext('td')
    werOI = table.find('td', text='WER (order independent)')
    werOIScore = werOI.findNext('td')
    
    os.remove(filename_gt)
    os.remove(filename_ocr)
    os.remove(output)
    return float(cerScore.text), float(werScore.text)   
    
    return cerScore.text, werScore.text

In [62]:
#for index, row in df.loc[df['set'].isin(['train', 'val'])].iterrows():
for index, row in df.iterrows():
    #print(index)
    if index%1000 == 0:
        print(index)
    doc_GT, doc_OCR = fuzzy_matching(row)
    #GT_matched_docs_list.append(matched_gt_sentences)
    #OCR_matched_docs_list.append(matched_ocr_sentences)
    if (row['set'] == 'val') or (row['set'] == 'test'):
        df.at[index, 'gt sentences matched'], df.at[index, 'ocr sentences matched'] = doc_GT, doc_OCR
    if row['set'] == 'test':
        df.at[index, 'CER matched sentences'], df.at[index, 'WER matched sentences'] = evaluation(index, row, doc_GT, doc_OCR)

#for index, row in df.loc[df['set'].isin(['test'])].iterrows():
    #print(index)
#    if index%1000 == 0:
#        print(index)
#    doc_GT, doc_OCR = fuzzy_matching(row)
    #GT_matched_docs_list.append(matched_gt_sentences)
    #OCR_matched_docs_list.append(matched_ocr_sentences)
#    df.at[index, 'gt sentences matched'], df.at[index, 'ocr sentences matched'] = doc_GT, doc_OCR
    #df.at[index, 'CER matched sentences'], df.at[index, 'WER matched sentences'] = evaluation(index, row, doc_GT, doc_OCR)
    

        


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000


In [71]:
# count sentences (longer than 4 words) in GT and OCR
# count words in sentences longer than 4 words

#meta_df = pd.DataFrame(columns=['identifier', 'sentences gt', 'sentences ocr', 'avg sentence length gt', ' avg sentence length ocr'])
meta_df = pd.DataFrame()

def sentence_count(row):
# count of sentences in GT >= 4 words
    sentences_gt = 0
    words_GT_sentences = []
    #characters_GT_sentences = []
    for k in row['gt text'].split('.'):
        k = [word for word in k.split(' ') if len(word) > 1]
        if len(k) >= 4:
            sentences_gt += 1
            words_GT_sentences.append(len(k))
            #characters_GT_sentences.append(len(' '.join(k)))
    # count of sentences in OCR >= 4 words
    sentences_ocr = 0
    words_OCR_sentences = []
    #characters_OCR_sentences = []
    for k in row['ocr text'].split('.'):
        k = [word for word in k.split(' ') if len(word) > 1]
        if len(k) >= 4:
            sentences_ocr += 1
            words_OCR_sentences.append(len(k))
            #characters_OCR_sentences.append(len(' '.join(k)))
    if len(words_GT_sentences) == 0:
        words_GT_sentences.append(0)
    if len(words_OCR_sentences) == 0:
        words_OCR_sentences.append(0)
    
    return sentences_gt, sentences_ocr, words_GT_sentences, words_OCR_sentences


#df['gt text'] = df['gt text'].apply(lambda x: preprocessing(x))

for index, row in df.iterrows(): 
    sentences_gt, sentences_ocr, words_GT_sentences, words_OCR_sentences = sentence_count(row)
    meta_df = meta_df.append({'identifier': row['identifier'], \
                              'sentences gt (fuzzy matched)': sentences_gt, 'sentences ocr (fuzzy matched)': sentences_ocr, \
                              'avg sentence length gt (fuzzy matched)': s.mean(words_GT_sentences), 'avg sentence length ocr (fuzzy matched)': s.mean(words_OCR_sentences), \
                              'max sentence length gt (fuzzy matched)': max(words_GT_sentences), 'max sentence length ocr (fuzzy matched)': max(words_OCR_sentences), \
                              'word count gt (fuzzy matched)': sum(words_GT_sentences), 'word count ocr (fuzzy matched)': sum(words_OCR_sentences)}, ignore_index = True)
    
df = pd.merge(left=df, right=meta_df, on='identifier')   

In [64]:
#df.to_csv('tussendoor_df_ER.csv')

In [72]:
#df2 = pd.read_csv('tussendoor_df_ER.csv')
df.head(1)

,Unnamed: 0,identifier,gt text,ocr text,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,...,CER matched sentences,WER matched sentences,avg sentence length gt (fuzzy matched),avg sentence length ocr (fuzzy matched),max sentence length gt (fuzzy matched),max sentence length ocr (fuzzy matched),sentences gt (fuzzy matched),sentences ocr (fuzzy matched),word count gt (fuzzy matched),word count ocr (fuzzy matched)
0,22517,ddd.010555149.mpeg21.a0002,wt soloturen den 28 dito,wt soloturen den 28 dito,57.53,114.86,113.51,85.71,42.25,0.14,...,NaN,NaN,5.0,5.0,5.0,5.0,1.0,1.0,5.0,5.0


In [73]:
df.columns

Index(['Unnamed: 0', 'identifier', 'gt text', 'ocr text', 'CER', 'WER',
       'WER (order independent)', 'dictionary lookup gt',
       'dictionary lookup ocr', 'jaccard_coefficient',
       'levenshtein_dist_normalized', 'source', 'word count gt',
       'word count ocr', 'year', 'century', 'old index', 'gt text org',
       'ocr text org', 'set', 'gt sentences matched', 'ocr sentences matched',
       'CER matched sentences', 'WER matched sentences',
       'avg sentence length gt (fuzzy matched)',
       'avg sentence length ocr (fuzzy matched)',
       'max sentence length gt (fuzzy matched)',
       'max sentence length ocr (fuzzy matched)',
       'sentences gt (fuzzy matched)', 'sentences ocr (fuzzy matched)',
       'word count gt (fuzzy matched)', 'word count ocr (fuzzy matched)'],
      dtype='object')

In [74]:
pd.set_option('display.max_columns', None)
#df

In [75]:
# sum of values
df.groupby('century').sum()

,Unnamed: 0,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,word count gt,word count ocr,old index,CER matched sentences,WER matched sentences,avg sentence length gt (fuzzy matched),avg sentence length ocr (fuzzy matched),max sentence length gt (fuzzy matched),max sentence length ocr (fuzzy matched),sentences gt (fuzzy matched),sentences ocr (fuzzy matched),word count gt (fuzzy matched),word count ocr (fuzzy matched)
century,,,,,,,,,,,,,,,,,,,,,
1600s,605782832,1255153.02,2528920.30,2330336.97,2.755678e+06,1753530.50,6276.398158,11738.650591,16257868,14864108,605855582,49167.80,165834.75,210246.466667,227472.466667,260197.0,273782.0,43470.0,46480.0,321388.0,357118.0
1700s,1284987,23676.87,48893.63,43489.96,1.453241e+05,136368.20,935.724665,240.738298,11131140,12094006,58750748,2224.54,7041.49,60625.193973,52397.386499,138781.0,118880.0,15212.0,15285.0,429338.0,379760.0
1800s,455002,13621.04,17616.52,10546.14,5.433854e+04,47984.05,398.767493,31.416851,49618121,52559527,23797627,832.22,2213.94,14067.522872,12643.913376,58733.0,53743.0,45992.0,48836.0,938414.0,908588.0
1900s,9809921,15208.24,28981.14,24579.25,1.515944e+05,143450.56,1243.443495,135.340352,5704994,5676973,64219046,2087.25,5109.26,30979.437012,29863.835202,92591.0,91758.0,52382.0,54127.0,889611.0,879512.0


In [76]:
# mean of values
df.groupby('century').mean()

,Unnamed: 0,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,word count gt,word count ocr,old index,CER matched sentences,WER matched sentences,avg sentence length gt (fuzzy matched),avg sentence length ocr (fuzzy matched),max sentence length gt (fuzzy matched),max sentence length ocr (fuzzy matched),sentences gt (fuzzy matched),sentences ocr (fuzzy matched),word count gt (fuzzy matched),word count ocr (fuzzy matched)
century,,,,,,,,,,,,,,,,,,,,,
1600s,17402.551910,36.057254,72.649247,66.944469,79.163402,50.374332,0.180304,0.337221,467.045906,427.006837,17404.641827,7.574765,25.548413,6.039830,6.534687,7.474777,7.865039,1.248779,1.335248,9.232634,10.259063
1700s,780.672539,14.384490,29.704514,26.421604,88.289247,82.848238,0.568484,0.148421,6762.539490,7347.512758,35693.042527,6.887121,21.800279,36.831831,31.833163,84.314095,72.223572,9.241798,9.286148,260.837181,230.716889
1800s,721.080824,21.586434,27.918415,16.713376,86.114960,76.044453,0.631961,0.072057,78634.106181,83295.605388,37714.147385,6.604921,17.570952,22.294014,20.037898,93.079239,85.171157,72.887480,77.394612,1487.185420,1439.917591
1900s,5733.443016,8.888510,16.938130,14.365430,88.599860,83.840187,0.726735,0.079100,3334.303916,3317.926943,37533.048510,6.138971,15.027235,18.106042,17.454024,54.115137,53.628288,30.614845,31.634717,519.936295,514.033898


In [77]:
# to find out longest sentence
print(max(list(df['max sentence length gt (fuzzy matched)'])))

381.0


In [78]:
# word alignment
def word_alignment(GT, OCR):
    shortest_list = min([GT, OCR], key=len)
    longest_list = max([GT, OCR], key=len)
    min_ratio = 0.65

    for i in range(len(longest_list)):
        match = False
        #i = -i-1
        # check if there is a direct match:
        try:
            if OCR[i] == '' and GT[i] == '':
                del OCR[i]
                del GT[i]
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= min_ratio:
                match = True
                continue
            elif match == False:
                # check if there is a match with a word before in OCR:
                try:
                    for j in range(1,10):
                        if i-j >= 0:
                            if j >= 5:
                                if (GT[i] >= 5) or (OCR[i-j] >= 5):
                                    pass
                                else:
                                    continue
                            if SequenceMatcher(None, GT[i], OCR[i-j]).ratio() >= min_ratio:
                                if not (GT[i] == '' and OCR[i-j] == ''):
                                    for k in range(0,j):
                                        OCR.insert(i-j, '')
                                    match = True
                                    continue
                        if i+j <= len(OCR):
                            if j >= 5:
                                if (GT[i] >= 5) or (OCR[i+j] >= 5):
                                    pass
                                else:
                                    continue
                            if SequenceMatcher(None, GT[i], OCR[i+j]).ratio() >= min_ratio:
                                if not (GT[i] == '' and OCR[i+j] == ''):
                                    for k in range(0,j):
                                        GT.insert(i, '')
                                    match = True
                                    continue
                except:
                    pass
        except IndexError:
            min([GT, OCR], key=len).append('')

        if match == False:
            try:
                if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= 0.30:
                    pass
            except:
                pass
            else:
                #print('bad match', GT[i], OCR[i])
                #print('before forwards match GT:', GT)
                #print('before forwards bad match OCR:', OCR)
                if i-1 >= 0:
                    GT.insert(i, '')
                else:
                    GT.insert(0, '')
                if i+1 <= len(GT):
                    OCR.insert(i+1, '')
                else:
                    GT.append('')
                #print('after forwards match GT:', GT)
                #print('after forwards bad match OCR:', OCR)
                
                continue
    
   #print('after forwards GT:', GT)
    #print('after forwards OCR:', OCR)
    
    while len(GT) != len(OCR):
        min([GT, OCR], key=len).append('')
    
    if True:
        
        shortest_list = min([GT, OCR], key=len)
        longest_list = max([GT, OCR], key=len)

        for i in range(len(longest_list)):
            i = -i-1
            #print('i:', i)
            try:

                if (OCR[i] == '') ^ (GT[i] == ''):

                    continue
                if OCR[i] == ' ' and GT[i] == '':
                    del OCR[i]
                    del GT[i]
                if SequenceMatcher(None, GT[i], OCR[i]).ratio() > 0.65:
     
                    continue
                else:
                    # check if there is a match with a word before in OCR:
                    try:
                        for j in range(1,len(GT)):
                            if j >= 10:
                                extra_ratio = 0.1
                            else:
                                extra_ratio = 0
                            if i-j >= -len(OCR):
                                if j >= 5:
                                    if (len(GT[i]) >= 5) or (len(OCR[i-j]) >=5):
                                        pass
                                    else:
                                        continue
                            if GT[i] == '' or OCR[i-j] == '':
                                continue
                            if SequenceMatcher(None, GT[i], OCR[i-j]).ratio() > 0.65 + extra_ratio:
                                for k in range(0,j):
                                    if i+k < 0:
                                        GT.insert(i+1, '')
                                    else: 
                                        GT.append('')
                                break
                            if i+j <= -1:
                                if j >= 5:
                                    if (GT[i] >= 5) or (OCR[i+j] >=5):
                                        pass
                                    else:
                                        continue
                            if GT[i] == '' or OCR[i+j] == '':
                                continue
                            if SequenceMatcher(None, GT[i], OCR[i+j]).ratio() > 0.65 + extra_ratio:
                                for k in range(0,j):
                                    GT.insert(i, '')

                                break
                    except:
                        pass

            except IndexError:
                min([GT, OCR], key=len).append('')

            # if there is no match:
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() > 0.30:
                pass
            else:
                #print('bad match', GT[i], OCR[i])
                #print('before backwards bad match GT:',  GT)
                #print('before backwards bad match OCR:',  OCR)
                if (OCR[i] == '') or (GT[i] == ''):
                    continue
                else:
                    GT.insert(i, '')
                    OCR.insert(i+1, '')
                #print('after backwards bad match GT:',  GT)
                #print('after backwards bad match OCR:',  OCR)
                
                
    #print('after backwards GT:',  GT)
    #print('after backwards OCR:',  OCR)
                    
    while len(GT) != len(OCR):
        min([GT, OCR], key=len).append('')


    for i in range(len(GT)-1):
        try:
            #print(i)
            #print(GT[i], OCR[i])
            if (GT[i] == '') and (OCR[i] == ''):
                del GT[i]
                del OCR[i]
        except:
            break
            
    try:
        if SequenceMatcher(None, GT[-1], OCR[-2]).ratio() >= 0.65 and ((GT[-1] != '') or ((OCR[-2] != ''))):
            #print('FOUND:', GT[-1], OCR[-2])
            OCR.insert(-2, '')
            GT.append('')
    except:
        pass
    try:
        if SequenceMatcher(None, GT[-2], OCR[-1]).ratio() >= 0.65 and ((GT[-2] != '') or ((OCR[-1] != ''))):
            #print('FOUND:', OCR[-1], GT[-2])
            GT.insert(-2, '')
            OCR.append('')
    except:
        pass
    
    for i in range(len(OCR)):
        if GT[i] == '' and OCR[i] == '':
            GT[i] = 'REMOVE'
            OCR[i] = 'REMOVE'

    GT = list(filter(lambda a: a != 'REMOVE', GT))
    OCR = list(filter(lambda a: a != 'REMOVE', OCR))
    
    
    #print('end GT:',  GT)
    #print('end OCR:',  OCR)
    return GT, OCR

In [79]:
df.columns

Index(['Unnamed: 0', 'identifier', 'gt text', 'ocr text', 'CER', 'WER',
       'WER (order independent)', 'dictionary lookup gt',
       'dictionary lookup ocr', 'jaccard_coefficient',
       'levenshtein_dist_normalized', 'source', 'word count gt',
       'word count ocr', 'year', 'century', 'old index', 'gt text org',
       'ocr text org', 'set', 'gt sentences matched', 'ocr sentences matched',
       'CER matched sentences', 'WER matched sentences',
       'avg sentence length gt (fuzzy matched)',
       'avg sentence length ocr (fuzzy matched)',
       'max sentence length gt (fuzzy matched)',
       'max sentence length ocr (fuzzy matched)',
       'sentences gt (fuzzy matched)', 'sentences ocr (fuzzy matched)',
       'word count gt (fuzzy matched)', 'word count ocr (fuzzy matched)'],
      dtype='object')

In [80]:
# verify word alignment quality
def verify_alignment(GT, OCR):
    
    if len(GT) != len(OCR):
        alignment_validation = False
        return alignment_validation
    
    min_word_length = 8 # the higher this number, the more likely a alignment will be seen as valid, thus higher chance at false positive, lower chance at false negative. 
    match_ratio = 0.70
    anchors_list = []
    for i in range(len(GT)):
        if len(GT[i]) >= min_word_length:  
            anchors_list.append(GT[i])
        if len(OCR[i]) >= min_word_length:
            anchors_list.append(OCR[i])
    #print(anchors_list)


    alignment_validation = True

    for anchor in anchors_list:
        #print('ANCHOR:', anchor)
        matches_GT = set()
        matches_OCR = set()
        match = False
        for i in range(len(GT)):
            if (SequenceMatcher(None, GT[i], anchor).ratio() >= 0.65) and (len(GT[i]) >= min_word_length or len(GT[i]) == 0):
                matches_GT.add(i)
                #print('added as GT match:', GT[i])
            if SequenceMatcher(None, OCR[i], anchor).ratio() >= 0.65 and (len(OCR[i]) >= min_word_length or len(OCR[i]) == 0):
                matches_OCR.add(i)
                #print('added as OCR match:', OCR[i])
        #print('anchor:', anchor)
        #print(matches_GT)
        #print(matches_OCR)
        if matches_GT == matches_OCR:
            match = True
        if match == False:
            match = True
            points_set1 = set() # counts when the matched opposite word is '' for OCR
            points_set2 = set() # counts when the matched opposite word is '' for GT
            for index in matches_GT.difference(matches_OCR): # elements in matches_GT that are not in matches_OCR
                if OCR[index] == '':
                    points_set1.add(index)
                if OCR[index] != '' and len(matches_OCR.difference(matches_GT)) != 0:
                    match = False
            for index in matches_OCR.difference(matches_GT): # elements in matches_OCR that are not in matches_GT
                if GT[index] == '':
                    points_set2.add(index)
                if GT[index] != '' and len(matches_GT.difference(matches_OCR)) != 0:
                    match = False
            if len(points_set1.intersection(points_set2)) != 0: # if an anchor words was match in both OCR and GT with an '', it's a bad match.
                match = False
        if match == False:
            for index in (matches_GT.difference(matches_OCR)):
                index += 1
                if index in matches_OCR.difference(matches_GT):
                    if (GT[index-1] == '' or OCR[index-1] == '') and (SequenceMatcher(None, GT[index], OCR[index]).ratio() >= match_ratio):
                        match = True
            for index in (matches_OCR.difference(matches_GT)):
                index += 1
                if index in matches_GT.difference(matches_OCR):
                    if (GT[index] == '' or OCR[index] == '') and (SequenceMatcher(None, GT[index-1], OCR[index-1]).ratio() >= match_ratio):
                        match = True

        if match == False:
            alignment_validation = False # bad alignment

    return alignment_validation

In [81]:


good_alignments_docs = []
bad_alignments_docs = []
aligned_OCR_sentences_docs = []
aligned_GT_sentences_docs = []

#df['aligned_GT_sentences'] = [np.nan] * len(df)
#df['aligned_OCR_sentences'] = [np.nan] * len(df)
#df['good_alignments'] = [np.nan] * len(df)
#df['bad_alignments'] = [np.nan] * len(df)

#for index, row in df.loc[df['set'].isin(['val','test'])].iterrows():
for index, row in df.iterrows():
    if (row['set'] == 'val') or (row['set'] == 'test'):
        GT_sentences = row['gt sentences matched'].split('.')
        OCR_sentences = row['ocr sentences matched'].split('.')
        good_alignments = 0
        bad_alignments = 0
        aligned_GT = []
        aligned_OCR = []
        for i in range(len(GT_sentences)):
            GT = GT_sentences[i].split(' ')
            OCR = OCR_sentences[i].split(' ')
            GT, OCR = word_alignment(GT, OCR)
            verification = verify_alignment(GT, OCR)
            if verification == False: # bad alignment
                GT = ['REMOVED'] * len(GT)
                bad_alignments += 1
            else:
                good_alignments += 1
            aligned_GT.append(GT)
            aligned_OCR.append(OCR)
        # an alignment has been made for one document
        #df.loc[index, 'aligned_GT_sentences'] = str(aligned_GT)
        #df.loc[index, 'aligned_OCR_sentences'] = str(aligned_OCR)
        #df.at[index, 'good alignments'] = good_alignments
        #df.at[index, 'bad alignments'] = bad_alignments
        good_alignments_docs.append(good_alignments)
        bad_alignments_docs.append(bad_alignments)
        aligned_OCR_sentences_docs.append(aligned_OCR)
        aligned_GT_sentences_docs.append(aligned_GT)
        
    else:
        #df.loc[index, 'aligned_GT_sentences'] = df.loc[index, 'aligned_OCR_sentences'] = df.at[index, 'good alignments'] = df.at[index, 'bad alignments'] = np.nan
        good_alignments_docs.append(np.nan)
        bad_alignments_docs.append(np.nan)
        aligned_OCR_sentences_docs.append(np.nan)
        aligned_GT_sentences_docs.append(np.nan)

    # this list should only apply to rows in the val and test set
df['aligned_GT_sentences'] = aligned_GT_sentences_docs
df['aligned_OCR_sentences'] = aligned_OCR_sentences_docs
df['good_alignments'] = good_alignments_docs
df['bad_alignments'] = bad_alignments_docs
df['good alignments percentage'] = df['good_alignments'] / (df['good_alignments'] + df['bad_alignments'])
#print('percentage of sentences in dataset that is well aligned:', good_alignments_percentage)
print(df[['good alignments percentage', 'century']].groupby('century').mean())

    
  

         good alignments percentage
century                            
1600s                      0.965870
1700s                      0.777526
1800s                      0.918942
1900s                      0.918161


In [82]:
#df['longest_streak'], df['avg_longest_streaks'] = df['avg_total_missing_words'] = df['avg_perc_missing_words '] = [np.nan] * len(df)
longest_streak_docs = []
avg_longest_streaks_docs = []
avg_total_missing_words_docs = []
avg_perc_missing_words_docs = [] 
print(len(longest_streak_docs))

for index, row in df.iterrows():
    if (row['set'] == 'val') or (row['set'] == 'test'):
        ocr_aligned_sentences = row['aligned_OCR_sentences']
        # analysing the word alignment
        # word alignment for each sentence
        longest_streaks = []
        total_missing_words = []
        perc_missing_words = []
        for sentence in ocr_aligned_sentences: # for each sentence
            streaks = []
            streak = 0
            for word in sentence: # for each word
                if word == '':
                    streak = streak + 1
                else:
                    streaks.append(streak)
                    streak = 0
            try:
                longest_streaks.append(max(streaks))# longest streak of missing word in sentence
                total_missing_words.append(sum(streaks)) # total of missing words in a sentence
                perc_missing_words.append(sum(streaks) / len(ocr_aligned_sentences))
            except ValueError:
                longest_streaks.append(0)# longest streak of missing word in sentence
                total_missing_words.append(0) # total of missing words in a sentence
                perc_missing_words.append(0)
        # statistics word alignment doc level:
        # find longest streak of all longest streaks:
        import statistics as s
        longest_streak = max(longest_streaks)
        avg_longest_streaks = s.mean(longest_streaks)
        avg_total_missing_words = s.mean(total_missing_words)
        avg_perc_missing_words = s.mean(perc_missing_words)
        longest_streak_docs.append(longest_streak)
        avg_longest_streaks_docs.append(avg_longest_streaks)
        avg_total_missing_words_docs.append(avg_total_missing_words)
        avg_perc_missing_words_docs.append(avg_perc_missing_words)
    elif (row['set'] == 'train'):
        longest_streak_docs.append(np.nan)
        avg_longest_streaks_docs.append(np.nan)
        avg_total_missing_words_docs.append(np.nan)
        avg_perc_missing_words_docs.append(np.nan)
   
df['longest_streak'] = longest_streak_docs
df['avg_longest_streaks'] = avg_longest_streaks_docs
df['avg_total_missing_words'] = avg_total_missing_words_docs
df['avg_perc_missing_words '] = avg_perc_missing_words_docs
          

0


In [83]:
pd.set_option('display.max_rows', None)
#df


In [84]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def find_names(text, names):
    NNP_tags = "ik je jij jou u hij zij ze het wij we jullie mij me hem haar ons hen hun".split(' ')
    text = word_tokenize(text)
    tags = nltk.pos_tag(text)
    
    for tag in tags:
        if (tag[1] == 'NNP'):
            # we want words that are NN(P)s, but are not in NNP_tags
            NN = tag[0]
            NN = NN.lower()
            if NN not in NNP_tags:
                names.append(NN)
    return names

# find proper pronouns
gt_names = []
ocr_names = []
# these both can look into train, val, and test set
for text in list(df['gt text']):
    gt_names_org = find_names(text, gt_names)
for text in list(df['ocr text']):
    ocr_names_org = find_names(text, ocr_names)

gt_names = []
for name in gt_names_org:
    if len(name) >= 5:
        gt_names.append(name)
ocr_names = []
for name in ocr_names_org:
    if len(name) >= 5:
        ocr_names.append(name)

[nltk_data] Downloading package punkt to /home/nvanthof/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nvanthof/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [85]:
# save dataframe
df.to_csv('preprocessed_df.csv', index=False)

In [86]:
# save lists:
with open("gt_names.txt", "wb") as fp:   #Pickling
    pickle.dump(gt_names, fp)
with open("ocr_names.txt", "wb") as fp:   #Pickling
    pickle.dump(ocr_names, fp)
#with open("test.txt", "rb") as fp:   # Unpickling
#    b = pickle.load(fp)